In [5]:
import numpy as np
import random
import time

USE_SKLEARN = True
X = None
y = None
try:
    from sklearn.datasets import make_classification
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_val_score
except Exception as e:
    USE_SKLEARN = False
    print("sklearn not available, using synthetic fitness fallback:", e)

if USE_SKLEARN:
    try:
        X, y = make_classification(n_samples=200, n_features=8, n_informative=6,
                                   n_redundant=0, n_repeated=0, n_classes=2,
                                   class_sep=1.5, flip_y=0.0, random_state=1)
    except Exception as e:
        USE_SKLEARN = False
        X = None; y = None
        print("failed to create dataset, falling back to synthetic fitness:", e)

def fitness_from_params(logC, logG):
    if USE_SKLEARN and X is not None and y is not None:
        try:
            C = 10 ** logC
            gamma = 10 ** logG
            clf = SVC(kernel='rbf', C=C, gamma=gamma, random_state=0)
            scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy', n_jobs=1)
            return float(scores.mean())
        except Exception as e:
            # if something goes wrong with sklearn during runtime, fall back
            print("sklearn runtime error, switching to fallback fitness:", e)
    # fallback synthetic multimodal fitness
    x, yy = float(logC), float(logG)
    val = np.exp(-((x - 1.5)**2 + (yy + 1.0)**2))
    val *= (0.6 + 0.4 * (np.cos(3*x) * np.cos(3*yy) + 1) / 2)
    return float(val)

def gwo(n_agents=16, max_iter=20, lb=(-2.0, -4.0), ub=(4.0, -0.5), seed=0, verbose=True):
    random.seed(seed)
    np.random.seed(seed)
    dim = 2
    lb = np.array(lb); ub = np.array(ub)
    positions = np.random.uniform(lb, ub, (n_agents, dim))
    fitness = np.array([fitness_from_params(p[0], p[1]) for p in positions])
    idx = fitness.argsort()[::-1]
    alpha_pos = positions[idx[0]].copy(); alpha_score = float(fitness[idx[0]])
    beta_pos  = positions[idx[1]].copy(); beta_score  = float(fitness[idx[1]])
    delta_pos = positions[idx[2]].copy(); delta_score = float(fitness[idx[2]])
    best_history = [alpha_score]

    start = time.time()
    for t in range(max_iter):
        a = 2.0 * (1.0 - t / float(max_iter))
        for i in range(n_agents):
            X_i = positions[i].copy()
            A1 = 2 * a * np.random.rand() - a
            C1 = 2 * np.random.rand()
            D_alpha = np.abs(C1 * alpha_pos - X_i)
            X1 = alpha_pos - A1 * D_alpha

            A2 = 2 * a * np.random.rand() - a
            C2 = 2 * np.random.rand()
            D_beta = np.abs(C2 * beta_pos - X_i)
            X2 = beta_pos - A2 * D_beta

            A3 = 2 * a * np.random.rand() - a
            C3 = 2 * np.random.rand()
            D_delta = np.abs(C3 * delta_pos - X_i)
            X3 = delta_pos - A3 * D_delta

            new_pos = (X1 + X2 + X3) / 3.0
            new_pos = np.minimum(np.maximum(new_pos, lb), ub)
            positions[i] = new_pos

        fitness = np.array([fitness_from_params(p[0], p[1]) for p in positions])
        idx = fitness.argsort()[::-1]
        if float(fitness[idx[0]]) > alpha_score:
            alpha_score = float(fitness[idx[0]])
            alpha_pos = positions[idx[0]].copy()
        if float(fitness[idx[1]]) > beta_score:
            beta_score = float(fitness[idx[1]])
            beta_pos = positions[idx[1]].copy()
        if float(fitness[idx[2]]) > delta_score:
            delta_score = float(fitness[idx[2]])
            delta_pos = positions[idx[2]].copy()
        best_history.append(alpha_score)

        if verbose:
            elapsed = time.time() - start
            print(f"iter {t+1:02d}/{max_iter}  best={alpha_score:.6f}  (logC={alpha_pos[0]:+.3f}, logG={alpha_pos[1]:+.3f})  elapsed={elapsed:.1f}s")

    best_logC, best_logG = alpha_pos[0], alpha_pos[1]
    best_C = 10 ** best_logC
    best_gamma = 10 ** best_logG
    best_acc = alpha_score
    return best_C, best_gamma, best_acc, best_history

if __name__ == "__main__":
    C, gamma, acc, hist = gwo(n_agents=12, max_iter=15, lb=(-2.0, -4.0), ub=(4.0, -0.5), seed=42, verbose=True)
    print("\nOptimal Parameter found")
    print(f"C = {C:.6f} , gamma = {gamma:.6f}")
    print(f"Best score = {acc:.6f}")


iter 01/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.1s
iter 02/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.2s
iter 03/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.3s
iter 04/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.4s
iter 05/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.5s
iter 06/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.6s
iter 07/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.7s
iter 08/15  best=0.980100  (logC=+2.392, logG=-1.905)  elapsed=0.7s
iter 09/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=0.8s
iter 10/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=0.9s
iter 11/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=1.0s
iter 12/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=1.1s
iter 13/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=1.2s
iter 14/15  best=0.985075  (logC=+1.938, logG=-2.009)  elapsed=1.3s
iter 15/15  best=0.985075  (logC=+1.938, logG=-2